<a href="https://colab.research.google.com/github/sjtalkar/DP-203-Azure-Data-Engineering-Notes/blob/main/SparkAndDeltaLake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# %%shell
# SCALA_VERSION=2.12.8 ALMOND_VERSION=0.3.0+16-548dc10f-SNAPSHOT
# curl -Lo coursier https://git.io/coursier-cli
# chmod +x coursier
# ./coursier bootstrap \
#     -r jitpack -r sonatype:snapshots \
#     -i user -I user:sh.almond:scala-kernel-api_$SCALA_VERSION:$ALMOND_VERSION \
#     sh.almond:scala-kernel_$SCALA_VERSION:$ALMOND_VERSION \
#     --sources --default=true \
#     -o almond-snapshot --embed-files=false
# rm coursier
# ./almond-snapshot --install --global --force
# rm almond-snapshot

In [3]:
# %%shell
# echo "{
#   \"language\" : \"scala\",
#   \"display_name\" : \"Scala\",
#   \"argv\" : [
#     \"bash\",
#     \"-c\",
#     \"env LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libpython3.6m.so:\$LD_PRELOAD java -jar /usr/local/share/jupyter/kernels/scala/launcher.jar --connection-file {connection_file}\"
#   ]
# }" > /usr/local/share/jupyter/kernels/scala/kernel.json

# Spark's version 3.0.0 is being used here so that Delta Lake Core works well with it. 

#### Versions are something that will plague you when you have to work with Spark and other libraries.

In [4]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!tar xf 'gdrive/My Drive/Databricks/spark-3.0.0-bin-hadoop3.2.tgz' 
!pip -q install findspark
!pip install ipython-sql

In [5]:
!pwd

/content


In [6]:
%ls 'gdrive/My Drive/Databricks/spark-3.0.0-bin-hadoop3.2.tgz'

'gdrive/My Drive/Databricks/spark-3.0.0-bin-hadoop3.2.tgz'


In [7]:
%ls /content

gdrive/  sample_data/  spark-3.0.0-bin-hadoop3.2/


In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'


In [9]:
import findspark
findspark.init()

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('delta_session').getOrCreate()

In [11]:
from pyspark.sql.types import *

In [12]:
# create database to house SQL tables
_ = spark.sql('CREATE DATABASE IF NOT EXISTS kkbox')

In [13]:
transaction_schema = StructType([
            StructField('msno', StringType()),
            StructField('payment_method_id', IntegerType()),
            StructField('payment_plan_days', IntegerType()),
            StructField('plan_list_price', IntegerType()),
            StructField('actual_amount_paid', IntegerType()),
            StructField('is_auto_renew', IntegerType()),
            StructField('transaction_date', DateType()),
            StructField('membership_expire_date', DateType()),
            StructField('is_cancel', IntegerType()),
            ])
#read data from csv
transactions = (
                spark
                  .read
                  .csv(
                      'gdrive/My Drive/Databricks/transactions.csv',
                       schema=transaction_schema,
                       header=True,
                       dateFormat = 'yyyyMMdd'
                  )
)

# Work with the data using DELTA

In [14]:
#persist in delta lake format - essentially a parquet file with additional features such as history and versioning
# This creates  folders called  G:\My Drive\Databricks\transactions\transaction_date=2015-01-01 ......
(
  transactions
    .write
    .format('delta')
    .partitionBy('transaction_date')
    .mode('overwrite')
    .save('gdrive/My Drive/Databricks/kkbox/transactions')    
)

In [15]:
spark.sql("""
    DROP  TABLE  IF EXISTS kkbox.transactions
""")

DataFrame[]

In [16]:
spark.sql("""
    CREATE TABLE  kkbox.transactions 
    USING DELTA
    LOCATION 'gdrive/My Drive/Databricks/kkbox/transactions'
""")

DataFrame[]

In [17]:
capture = spark.sql('''
SELECT * FROM  kkbox.transactions LIMIT 10
''')

In [18]:
capture.show()

+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|+2HVGotjiE2ofWgVp...|               41|               30|             99|                99|            1|      2017-03-31|            2017-04-30|        0|
|+5Yo2rxxC+x+kIYl4...|               34|               30|            149|               149|            1|      2017-03-31|            2017-04-30|        0|
|+BForXQeVUWKHbTM/...|               39|               30|            149|               149|            1|      2017-03-31|            2017-05-16|        0|
|+L/XrtIg0DD9ku+ik...|               33|            

In [19]:

# members dataset schema
member_schema = StructType([
  StructField('msno', StringType()),
  StructField('city', IntegerType()),
  StructField('bd', IntegerType()),
  StructField('gender', StringType()),
  StructField('registered_via', IntegerType()),
  StructField('registration_init_time', DateType())
  ])

# read data from csv
members = (
  spark
    .read
    .csv(
      'gdrive/My Drive/Databricks/members.csv',
      schema=member_schema,
      header=True,
      dateFormat='yyyyMMdd'
      )
    )



In [20]:
# persist in delta lake format
(
  members
    .write
    .format('delta')
    .mode('overwrite')
    .save('gdrive/My Drive/Databricks/kkbox/members')
  )


In [21]:

  # create table object to make delta lake queriable
_ = spark.sql('''
    CREATE TABLE kkbox.members 
    USING DELTA 
    LOCATION 'gdrive/My Drive/Databricks/kkbox/members'
    ''')

In [22]:

#Load User Logs Table

_ = spark.sql('DROP TABLE IF EXISTS kkbox.user_logs')

# drop any old delta lake files that might have been created
#shutil.rmtree('gdrive/My Drive/Databricks/kkbox/user_logs', ignore_errors=True)

# user logs dataset schema
user_logs_schema = StructType([ 
  StructField('msno', StringType()),
  StructField('date', DateType()),
  StructField('num_25', IntegerType()),
  StructField('num_50', IntegerType()),
  StructField('num_75', IntegerType()),
  StructField('num_985', IntegerType()),
  StructField('num_100', IntegerType()),
  StructField('num_uniq', IntegerType()),
  StructField('total_secs', FloatType())  
  ])

# read data from csv
user_logs = (
  spark
    .read
    .csv(
      'gdrive/My Drive/Databricks/user_logs.csv',
      schema=user_logs_schema,
      header=True,
      dateFormat='yyyyMMdd'
      )
    )


In [23]:

# persist in delta lake format
( user_logs
    .write
    .format('delta')
    .partitionBy('date')
    .mode('overwrite')
    .save('gdrive/My Drive/Databricks/kkbox/user_logs')
  )

In [24]:


# create table object to make delta lake queriable
_ = spark.sql('''
  CREATE TABLE IF NOT EXISTS kkbox.user_logs
  USING DELTA 
  LOCATION 'gdrive/My Drive/Databricks/kkbox/user_logs'
  ''')

In [25]:
capture = spark.sql('''
SELECT * FROM  kkbox.members LIMIT 10
''')

In [26]:
capture.show()

+--------------------+----+---+------+--------------+----------------------+
|                msno|city| bd|gender|registered_via|registration_init_time|
+--------------------+----+---+------+--------------+----------------------+
|Rb9UwLQTrxzBVwCB6...|   1|  0|  null|            11|            2011-09-11|
|+tJonkh+O1CA796Fm...|   1|  0|  null|             7|            2011-09-14|
|cV358ssn7a0f7jZOw...|   1|  0|  null|            11|            2011-09-15|
|9bzDeJP6sQodK73K5...|   1|  0|  null|            11|            2011-09-15|
|WFLY3s7z4EZsieHCt...|   6| 32|female|             9|            2011-09-15|
|yLkV2gbZ4GLFwqTOX...|   4| 30|  male|             9|            2011-09-16|
|jNCGK78YkTyId3H3w...|   1|  0|  null|             7|            2011-09-16|
|WH5Jq4mgtfUFXh2yz...|   5| 34|  male|             9|            2011-09-16|
|tKmbR4X5VXjHmxERr...|   5| 19|  male|             9|            2011-09-17|
|I0yFvqMoNkM8ZNHb6...|  13| 63|  male|             9|            2011-09-18|

# Get Training Data from CSV

In [27]:
train_schema = StructType([
            StructField('msno', StringType()),
            StructField('is_churn', IntegerType()),
            ])
#read data from csv
train = (
                spark
                  .read
                  .csv(
                      'gdrive/My Drive/Databricks/train_v2.csv',
                       schema=train_schema,
                       header=True
                  )
)

In [28]:
(
  train
    .write
    .format('delta')
    .mode('overwrite')
    .save('gdrive/My Drive/Databricks/kkbox/train')    
)

In [32]:

# create table object to make delta lake queriable
_ = spark.sql('''
  CREATE TABLE IF NOT EXISTS kkbox.train
  USING DELTA 
  LOCATION 'gdrive/My Drive/Databricks/kkbox/train'
  ''')

In [33]:
capture = spark.sql('''
SELECT * FROM  kkbox.train LIMIT 10
''')

In [35]:
capture.show()

+--------------------+--------+
|                msno|is_churn|
+--------------------+--------+
|ugx0CjOMzazClkFzU...|       1|
|f/NmvEzHfhINFEYZT...|       1|
|zLo9f73nGGT1p21lt...|       1|
|8iF/+8HY8lJKFrTc7...|       1|
|K6fja4+jmoZ5xG6By...|       1|
|ibIHVYBqxGwrSExE6...|       1|
|kVmM8X4iBPCOfK/m1...|       1|
|moRTKhKIDvb+C8ZHO...|       1|
|dW/tPZMDh2Oz/ksdu...|       1|
|otEcMhAX3mU4gumUS...|       1|
+--------------------+--------+



In [37]:
train.head(5)

[Row(msno='ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=', is_churn=1),
 Row(msno='f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=', is_churn=1),
 Row(msno='zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=', is_churn=1),
 Row(msno='8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=', is_churn=1),
 Row(msno='K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=', is_churn=1)]

# Set up SQL magic

In [39]:
!pip install sparksql-magic

     |████████████████████████████████| 281.3 MB 33 kB/s 
     |████████████████████████████████| 790 kB 36.9 MB/s 
     |████████████████████████████████| 374 kB 64.9 MB/s 
     |████████████████████████████████| 198 kB 51.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=3e8af06fb1b31a3210d4df7d63c5ddd54ec5cfd41d3a63476021c07f2a8a198c
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the so

In [41]:
%load_ext sparksql_magic


In [42]:
%config SparkSql.max_num_rows = 20

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Config option `max_num_rows` not recognized by `SparkSql`.
  """Entry point for launching an IPython kernel.


In [45]:
%%sparksql
select * from kkbox.transactions limit 20

msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
+2HVGotjiE2ofWgVp37vLKGzJgBiUkdKRiBlf/Wji90=,41,30,99,99,1,2017-03-31,2017-04-30,0
+5Yo2rxxC+x+kIYl4to0D1GTrKacKcaGG0sV38NLIn4=,34,30,149,149,1,2017-03-31,2017-04-30,0
+BForXQeVUWKHbTM/nXSRGfhhzcWmfNfvz+HbE77Pjc=,39,30,149,149,1,2017-03-31,2017-05-16,0
+L/XrtIg0DD9ku+ik959EtrnRQfLyf/P6GD1Z8HbkP4=,33,30,149,149,1,2017-03-31,2017-04-30,0
+NFlZlsTdfUWAxDiEZ08D/ux0o+uY3ps1+6wy4I9ybg=,39,30,149,149,1,2017-03-31,2017-05-16,0
+OQ8X0rnnrPOUvtABzE+MRE9zqSjTzSrI42qawMQ2uU=,34,30,149,149,1,2017-03-31,2017-04-30,0
+SzJCXTBfFx9+tps9joOuArGrOsGoCD7Utj9aon7X+M=,39,30,149,149,1,2017-03-31,2017-05-12,0
+ZLD2EVyD7TQs3gUwTUeWDce5ZxUOSNpAXFd0roNO9o=,39,30,149,149,1,2017-03-31,2017-05-16,0
+bnLGdIZW5uxgyUA26TXwDMnTHTuDlbj4mFEAQtmmDo=,34,30,149,149,1,2017-03-31,2017-04-30,0
+hE9XWSj68wdPEaE8nyb4RN7i2xRRcN0VG13OUEJ4PE=,36,30,180,180,1,2017-03-31,2017-04-30,0


# Using Scala For labels

In [29]:
# Step 2: Acquire Churn Labels
# To build our model, we will need to identify which customers have churned within two periods of interest.
# These periods are February 2017 and March 2017. We will train our model to predict churn in February 2017 and then evaluate our model's ability 
# to predict churn in March 2017, making these our training and testing datasets, respectively.

# Per instructions provided in the Kaggle competition, a KKBox subscriber is not identified as churned until he or she fails to renew their 
#subscription 30-days following its expiration. Most subscriptions are themselves on a 30-day renewal schedule (though some subscriptions renew on significantly longer cycles). This means that identifying churn involves a sequential walk through the customer data, looking for renewal gaps that would indicate a customer churned on a prior expiration date.

# While the competition makes available pre-labeled training and testing datasets, train.csv and train_v2.csv, respectively, several past 
#participants have noted that these datasets should be regenerated. A Scala script for doing so is provided by KKBox. Modifying the script for this environment, we might regenerate our training and test datasets as follows:

In [30]:
# %scala

# import java.time.{LocalDate}
# import java.time.format.DateTimeFormatter
# import java.time.temporal.ChronoUnit

# import org.apache.spark.sql.{Row, SparkSession}
# import org.apache.spark.sql.functions._
# import scala.collection.mutable

# def calculateLastday(wrappedArray: mutable.WrappedArray[Row]) :String ={
#   val orderedList = wrappedArray.sortWith((x:Row, y:Row) => {
#     if(x.getAs[String]("transaction_date") != y.getAs[String]("transaction_date")) {
#       x.getAs[String]("transaction_date") < y.getAs[String]("transaction_date")
#     } else {
      
#       val x_sig = x.getAs[String]("plan_list_price") +
#         x.getAs[String]("payment_plan_days") +
#         x.getAs[String]("payment_method_id")

#       val y_sig = y.getAs[String]("plan_list_price") +
#         y.getAs[String]("payment_plan_days") +
#         y.getAs[String]("payment_method_id")

#       //same plan, always subscribe then unsubscribe
#       if(x_sig != y_sig) {
#         x_sig > y_sig
#       } else {
#         if(x.getAs[String]("is_cancel")== "1" && y.getAs[String]("is_cancel") == "1") {
#           //multiple cancel, consecutive cancels should only put the expiration date earlier
#           x.getAs[String]("membership_expire_date") > y.getAs[String]("membership_expire_date")
#         } else if(x.getAs[String]("is_cancel")== "0" && y.getAs[String]("is_cancel") == "0") {
#           //multiple renewal, expiration date keeps extending
#           x.getAs[String]("membership_expire_date") < y.getAs[String]("membership_expire_date")
#         } else {
#           //same day same plan transaction: subscription preceeds cancellation
#           x.getAs[String]("is_cancel") < y.getAs[String]("is_cancel")
#         }
#       }
#     }
#   })
#   orderedList.last.getAs[String]("membership_expire_date")
# }

# def calculateRenewalGap(log:mutable.WrappedArray[Row], lastExpiration: String): Int = {
#   val orderedDates = log.sortWith((x:Row, y:Row) => {
#     if(x.getAs[String]("transaction_date") != y.getAs[String]("transaction_date")) {
#       x.getAs[String]("transaction_date") < y.getAs[String]("transaction_date")
#     } else {
      
#       val x_sig = x.getAs[String]("plan_list_price") +
#         x.getAs[String]("payment_plan_days") +
#         x.getAs[String]("payment_method_id")

#       val y_sig = y.getAs[String]("plan_list_price") +
#         y.getAs[String]("payment_plan_days") +
#         y.getAs[String]("payment_method_id")

#       //same data same plan transaction, assumption: subscribe then unsubscribe
#       if(x_sig != y_sig) {
#         x_sig > y_sig
#       } else {
#         if(x.getAs[String]("is_cancel")== "1" && y.getAs[String]("is_cancel") == "1") {
#           //multiple cancel of same plan, consecutive cancels should only put the expiration date earlier
#           x.getAs[String]("membership_expire_date") > y.getAs[String]("membership_expire_date")
#         } else if(x.getAs[String]("is_cancel")== "0" && y.getAs[String]("is_cancel") == "0") {
#           //multiple renewal, expire date keep extending
#           x.getAs[String]("membership_expire_date") < y.getAs[String]("membership_expire_date")
#         } else {
#           //same date cancel should follow subscription
#           x.getAs[String]("is_cancel") < y.getAs[String]("is_cancel")
#         }
#       }
#     }
#   })

#   //Search for the first subscription after expiration
#   //If active cancel is the first action, find the gap between the cancellation and renewal
#   val formatter = DateTimeFormatter.ofPattern("yyyy-MM-dd")
#   var lastExpireDate = LocalDate.parse(s"${lastExpiration.substring(0,4)}-${lastExpiration.substring(4,6)}-${lastExpiration.substring(6,8)}", formatter)
#   var gap = 9999
#   for(
#     date <- orderedDates
#     if gap == 9999
#   ) {
#     val transString = date.getAs[String]("transaction_date")
#     val transDate = LocalDate.parse(s"${transString.substring(0,4)}-${transString.substring(4,6)}-${transString.substring(6,8)}", formatter)
#     val expireString = date.getAs[String]("membership_expire_date")
#     val expireDate = LocalDate.parse(s"${expireString.substring(0,4)}-${expireString.substring(4,6)}-${expireString.substring(6,8)}", formatter)
#     val isCancel = date.getAs[String]("is_cancel")

#     if(isCancel == "1") {
#       if(expireDate.isBefore(lastExpireDate)) {
#         lastExpireDate = expireDate
#       }
#     } else {
#       gap = ChronoUnit.DAYS.between(lastExpireDate, transDate).toInt
#     }
#   }
#   gap
# }

# val data = spark
#   .read
#   .option("header", value = true)
#   .csv("/mnt/kkbox/transactions/")

# val historyCutoff = "20170131"

# val historyData = data.filter(col("transaction_date")>="20170101" and col("transaction_date")<=lit(historyCutoff))
# val futureData = data.filter(col("transaction_date") > lit(historyCutoff))

# val calculateLastdayUDF = udf(calculateLastday _)
# val userExpire = historyData
#   .groupBy("msno")
#   .agg(
#     calculateLastdayUDF(
#       collect_list(
#         struct(
#           col("payment_method_id"),
#           col("payment_plan_days"),
#           col("plan_list_price"),
#           col("transaction_date"),
#           col("membership_expire_date"),
#           col("is_cancel")
#         )
#       )
#     ).alias("last_expire")
#   )

# val predictionCandidates = userExpire
#   .filter(
#     col("last_expire") >= "20170201" and col("last_expire") <= "20170228"
#   )
#   .select("msno", "last_expire")


# val joinedData = predictionCandidates
#   .join(futureData,Seq("msno"), "left_outer")

# val noActivity = joinedData
#   .filter(col("payment_method_id").isNull)
#   .withColumn("is_churn", lit(1))


# val calculateRenewalGapUDF = udf(calculateRenewalGap _)
# val renewals = joinedData
#   .filter(col("payment_method_id").isNotNull)
#   .groupBy("msno", "last_expire")
#   .agg(
#     calculateRenewalGapUDF(
#       collect_list(
#         struct(
#           col("payment_method_id"),
#           col("payment_plan_days"),
#           col("plan_list_price"),
#           col("transaction_date"),
#           col("membership_expire_date"),
#           col("is_cancel")
#         )
#       ),
#       col("last_expire")
#     ).alias("gap")
#   )

# val validRenewals = renewals.filter(col("gap") < 30)
#   .withColumn("is_churn", lit(0))
# val lateRenewals = renewals.filter(col("gap") >= 30)
#   .withColumn("is_churn", lit(1))

# val resultSet = validRenewals
#   .select("msno","is_churn")
#   .union(
#     lateRenewals
#       .select("msno","is_churn")
#       .union(
#         noActivity.select("msno","is_churn")
#       )
#   )

# resultSet.write.format("delta").mode("overwrite").save("/mnt/kkbox/silver/train/")

In [31]:
#https://pixiedust.github.io/pixiedust/install.html